In [9]:
import pandas as pd
import json

# Load Excel sheets
dashboard = pd.ExcelFile("data.xlsx")
data = pd.read_excel(dashboard, "Statistics")
aes = pd.read_excel(dashboard, 'AEs')
referrals = pd.read_excel(dashboard, 'Referrals')

# Fill missing values and convert types
data.fillna(0, inplace=True)
aes.fillna(0, inplace=True)
referrals.fillna(0, inplace=True)

data['year'] = data['year'].astype(int)
data['recordingMonth'] = data['recordingMonth'].astype(str)
aes['AERecordingYear'] = aes['AERecordingYear'].astype(int)
aes['AERecordingMonth'] = aes['AERecordingMonth'].astype(str)
referrals['ReferrelRecordingYear'] = referrals['ReferrelRecordingYear'].astype(int)
referrals['ReferrelRecordingMonth'] = referrals['ReferrelRecordingMonth'].astype(str)

# Process rows and build JSON
def process_row(row):
    row_data = row.to_dict()
    if int(pd.to_numeric(row.get('totalAES', 0), errors='coerce')) > 0:
        recordedAEs = aes[
            (aes['AERecordingSite'] == row['facilityName']) &
            (aes['AERecordingMonth'] == row['recordingMonth']) &
            (aes['AERecordingYear'] == row['year'])
        ]

        recordedAEs = recordedAEs[recordedAEs['ae_classification']!=0]

        row_data['matchingAES'] = recordedAEs.to_dict(orient='records')
    else:
        row_data['matchingAES'] = []
            
    if int(pd.to_numeric(row.get('referralToOtherServices', 0), errors='coerce')) > 0:
        otherReferrals = referrals[
            (referrals['ReferrelRecordingSite'] == row['facilityName']) &
            (referrals['ReferrelRecordingMonth'] == row['recordingMonth']) &
            (referrals['ReferrelRecordingYear'] == row['year'])
        ]

        otherReferrals = otherReferrals[otherReferrals['services_referred']!=0]
        # print(otherReferrals)
        row_data['otherReferrals'] = otherReferrals.to_dict(orient='records')
    else:
        print("no data")
        row_data['otherReferrals'] = []

    return row_data

json_data = data.apply(process_row, axis=1).tolist()

# Save to JSON
output_file = "data.json"
with open(output_file, "w") as file:
    json.dump(json_data, file, indent=4)

# Print the JSON
print(json.dumps(json_data, indent=4))


no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
no data
[
    {
        "AERecordingMonth": "December",
        "AERecordingSite": "Svita Clinic",
        "AERecordingYear": 2024.0,
        "District": "MBERENGWA",
        "ReferrelRecordingMonth": "December",
        "ReferrelRecordingSite": "Svita Clinic",
        "ReferrelRecordingYear": 2024.0,
        "Site_Name": "MUSUME - 101157 - MISSION HOSPITAL",
        "TotalMCsBYMethod": 43,
        "aeComments": "Client is healing",
        "ae_classification": "Severe",
        "ae_type_code": "COA",
        "checkMonthValidation": 1.0,
        "checkYearValidation": 1.0,
        "checkifFacilityHasBeenSaved": 0,
        "circumcising_cadre": "Nurse",
        "client_age": 16.0,
        "date_ae_identified": "2024-12-07",
        "facilityName": "Svita Clinic",
        "followUpTotal": 0.0,
        "fu15-19": 12,
        "fu20-24": 0.0,
        "fu25-29": 15.0,
        "fu30-34": 16.0,
        "fu35-39": 0

In [10]:
aes

,AERecordingMonth,AERecordingSite,AERecordingYear,District,cancelthisAEReport,aeComments,ae_classification,ae_type_code,circumcising_cadre,client_age,date_ae_identified,mcMethod,vmmc_number
0,November,Krima Clinic,2024,GOKWE SOUTH,0.0,Client healing,Severe,COA,Nurse,25,2024-11-10,Surgical,467233
1,November,Krima Clinic,2024,GOKWE SOUTH,0.0,Client healing,Moderate,BL,Nurse,18,2024-11-08,Surgical,245555
2,November,Mbembeswana,2024,MATOBO,0.0,Client is healing,Severe,COA,0,25,2024-10-03,Surgical,233446
3,November,Mbembeswana,2024,MATOBO,0.0,Client is healing,Moderate,SSE,Nurse,35,2024-10-03,Surgical,3467
4,0,0,0,MATOBO,0.0,Client Healimg at home,Severe,COA,Nurse,25,2024-09-03,Surgical,234455
5,0,0,0,MATOBO,0.0,Client Healing,Moderate,SSE,Nurse,23,2024-12-03,Surgical,578988
6,December,Svita Clinic,2024,MBERENGWA,0.0,Client is healing,Severe,COA,Nurse,16,2024-12-07,Surgical,585858
7,December,Musume Hospital,2024,MBERENGWA,0.0,Client is healing,Moderate,COA,Nurse,25,2024-12-18,Surgical,9857
8,December,Chipuriro,2024,GURUVE,0.0,Client is healing,Severe,COA,Nurse,25,2024-12-06,Surgical,8886
9,December,Guruve District Hospital,2024,GURUVE,0.0,Client is healing,Moderate,COA,Doctor,25,2024-12-02,Surgical,456
